# Interactive Travel Planning Assistant Using LangChain and APIs

*This notebook demonstrates the implementation of a travel planner assistant. By leveraging LangChain agents, it integrates APIs to fetch location coordinates, weather information, nearby places, and more. Additionally, it creates maps with clustered markers and saves itineraries for easy reference.*

### Instructions
Before running this notebook, ensure the following steps are completed:

#### Required API Keys:

OpenAI API Key (OPENAI_API_KEY): For GPT-4 agent functionality.

Google Maps API Key (GOOGLE_MAPS_API): For fetching location data and nearby places.

OpenWeatherMap API Key (OPENWEATHERMAP_API_KEY): For fetching weather data.

#### Setup openAPIkey.env File:

Create a openAPIkey.env file in the same directory as the notebook.
Add the following variables:



In [1]:
# OPENAI_API_KEY=your_openai_api_key
# GOOGLE_MAPS_API=your_google_maps_api_key
# OPENWEATHERMAP_API_KEY=your_openweathermap_api_key

### Notebook Code

In [2]:
# Install necessary libraries
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-community langchain-openai tavily-python

# Load environment variables
import os
from dotenv import load_dotenv

# Load API keys from .env file
dotenv_path = 'openAPIkey.env'
load_dotenv(dotenv_path)

# Verify API keys are loaded
print("OpenAI API Key Loaded:", os.environ.get('OPENAI_API_KEY') is not None)
print("Google Maps API Key Loaded:", os.environ.get('GOOGLE_MAPS_API') is not None)
print("OpenWeatherMap API Key Loaded:", os.environ.get('OPENWEATHERMAP_API_KEY') is not None)


### Define Tools

In [3]:
import requests
import logging
from typing import Dict, List, Any
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Environment variables
API_KEY = os.environ.get("GOOGLE_MAPS_API")
WEATHER_API_KEY = os.environ.get("OPENWEATHERMAP_API_KEY")

# Tool: Fetch Coordinates
def fetch_coordinates(location: str) -> Dict[str, float]:
    try:
        url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={API_KEY}"
        response = requests.get(url).json()
        if response.get("status") == "OK":
            location_data = response["results"][0]["geometry"]["location"]
            return {"latitude": location_data["lat"], "longitude": location_data["lng"]}
        return {}
    except Exception as e:
        logging.error(f"Error fetching coordinates: {e}")
        return {}

# Tool: Fetch Weather
def get_weather(location: str) -> str:
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather"
        params = {"q": location, "appid": WEATHER_API_KEY, "units": "metric"}
        response = requests.get(url, params=params).json()
        if response.get("cod") == 200:
            return f"{response['name']}: {response['weather'][0]['description']} at {response['main']['temp']}°C"
        return "Weather information unavailable."
    except Exception as e:
        logging.error(f"Error fetching weather: {e}")
        return "Weather information unavailable."

# Tool: Fetch Nearby Places
def fetch_nearby_places(lat: float, lng: float, radius: int, category: str) -> List[Dict[str, Any]]:
    try:
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            "location": f"{lat},{lng}",
            "radius": radius * 1000,
            "type": category,
            "key": API_KEY,
        }
        response = requests.get(url, params=params).json()
        if response.get("status") == "OK":
            return response.get("results", [])
        return []
    except Exception as e:
        logging.error(f"Error fetching nearby places: {e}")
        return []

# Tool: Currency Conversion
def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    try:
        url = f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
        response = requests.get(url).json()
        rate = response['rates'].get(to_currency)
        if rate:
            return f"{amount} {from_currency} = {amount * rate:.2f} {to_currency}"
        return "Currency conversion unavailable."
    except Exception as e:
        logging.error(f"Error converting currency: {e}")
        return "Currency conversion unavailable."

# Tool: Get Country and Currency
def get_country_and_currency(lat: float, lng: float) -> Dict[str, str]:
    try:
        url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={API_KEY}"
        response = requests.get(url).json()
        if response.get("status") == "OK":
            country = [comp['long_name'] for comp in response['results'][0]['address_components'] if "country" in comp['types']]
            country_name = country[0] if country else "Unknown"
            return {"country_name": country_name, "currency": "USD"}
        return {"country_name": "Unknown", "currency": "USD"}
    except Exception as e:
        logging.error(f"Error fetching country and currency: {e}")
        return {"country_name": "Unknown", "currency": "USD"}

# Tool: Create Map
def create_map(lat: float, lng: float, places: List[Dict[str, Any]]) -> str:
    try:
        import folium
        from folium.plugins import MarkerCluster

        map_object = folium.Map(location=[lat, lng], zoom_start=13)
        cluster = MarkerCluster().add_to(map_object)
        for place in places:
            folium.Marker(
                location=[place["geometry"]["location"]["lat"], place["geometry"]["location"]["lng"]],
                tooltip=place["name"]
            ).add_to(cluster)
        filename = "map.html"
        map_object.save(filename)
        return filename
    except Exception as e:
        logging.error(f"Error creating map: {e}")
        return "Map creation failed."

# Tool: Save Itinerary
def save_itinerary(itinerary: List[Dict[str, Any]]) -> str:
    try:
        import json
        filename = "itinerary.json"
        with open(filename, "w") as file:
            json.dump(itinerary, file)
        return filename
    except Exception as e:
        logging.error(f"Error saving itinerary: {e}")
        return "Itinerary saving failed."




*   Fetching Coordinates: Converts location name to latitude and longitude.
*   Getting Weather: Retrieves current weather details using OpenWeatherMap.
*   Finding Nearby Places: Finds nearby places based on category (e.g., restaurants, parks).
*   Creating a Map: Visualizes places on an interactive map.
*   Saving Itinerary: Saves details into a JSON file for future use.

In [4]:
### Agent Setup
# Define tools for the agent
tools = [
    Tool(name="weather_tool", func=get_weather, description="Fetch the weather for a location."),
    Tool(name="currency_conversion_tool", func=convert_currency, description="Convert currencies."),
    Tool(name="fetch_coordinates_tool", func=fetch_coordinates, description="Fetch coordinates for a location."),
    Tool(name="get_country_and_currency_tool", func=get_country_and_currency, description="Fetch country and currency info."),
    Tool(name="fetch_nearby_places_tool", func=fetch_nearby_places, description="Fetch nearby places."),
    Tool(name="create_map_tool", func=create_map, description="Create a map of places."),
    Tool(name="save_itinerary_tool", func=save_itinerary, description="Save the itinerary to a file.")
]

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_template("""
You are a travel assistant. Use the following tools:
1. Fetch coordinates for the location.
2. Get weather details.
3. Convert currency.
4. Fetch country and currency details.
5. Fetch nearby places.
6. Create a map of the places.
7. Save the itinerary.,
{agent_scratchpad}
""")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


### Workflow Execution

In [5]:
async def run_workflow(location: str, radius: int, category: str, amount: float):
    coordinates = fetch_coordinates(location)
    weather = get_weather(location)
    country_currency = get_country_and_currency(coordinates["latitude"], coordinates["longitude"])
    places = fetch_nearby_places(coordinates["latitude"], coordinates["longitude"], radius, category)
    map_file = create_map(coordinates["latitude"], coordinates["longitude"], places)
    converted_amount = convert_currency(amount, "USD", country_currency["currency"])
    itinerary = [{"name": place["name"], "rating": place.get("rating", "N/A")} for place in places]
    save_itinerary(itinerary)

    print(f"Coordinates: {coordinates}")
    print(f"Weather: {weather}")
    print(f"Country & Currency: {country_currency}")
    print(f"Nearby Places: {places[:5]}")
    print(f"Map File: {map_file}")
    print(f"Converted Amount: {converted_amount}")


### Run Workflow

In [6]:
import nest_asyncio
nest_asyncio.apply()

await run_workflow("Hyderabad", radius=5, category="restaurant", amount=5000)


Coordinates: {'latitude': 17.406498, 'longitude': 78.47724389999999}
Weather: Hyderabad: haze at 28.23°C
Country & Currency: {'country_name': 'India', 'currency': 'USD'}
Nearby Places: [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 17.4241132, 'lng': 78.4569356}, 'viewport': {'northeast': {'lat': 17.4253438802915, 'lng': 78.4584008302915}, 'southwest': {'lat': 17.4226459197085, 'lng': 78.45570286970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hotel Inner Circle', 'opening_hours': {'open_now': True}, 'photos': [{'height': 720, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101743948211697243081">Hotel Inner Circle</a>'], 'photo_reference': 'AWYs27z6szjsR6Y74uMS3hYcdbeAv6TuoKHn4UBudkafPqLSr5LlJbpk2KLja2SVXwVUCQbObusJzloAFMn4v54kmb7OiGhrNbp8ka10yDHnPLNXpl88mEGZH2Z

### Define Tools Using StructuredTool

### What is a StructuredTool?

The `StructuredTool` in LangChain is a powerful abstraction that allows you to define tools with structured inputs and outputs. Instead of dealing with unstructured data, `StructuredTool` enforces clear data validation using models like `pydantic.BaseModel`. This ensures that the input parameters and output formats are well-defined, making your tools more robust and less error-prone.

**Key Advantages:**
- Enforces input and output validation using schemas.
- Ensures better integration with agents, as the expected format is explicit.
- Prevents runtime errors caused by missing or invalid data.

In this notebook, we use `StructuredTool` to define and manage API interactions, such as fetching coordinates, weather, nearby places, and more.


### Example: Fetching Coordinates with StructuredTool

In the travel assistant workflow, we define a `StructuredTool` to fetch the coordinates of a location. Here's how we do it:

1. Define the input and output models using `pydantic.BaseModel`.
2. Implement the core function (`fetch_coordinates`) to call the Google Maps API.
3. Use `StructuredTool.from_function` to create the tool.

**Code Example:**
```python
class CoordinatesInput(BaseModel):
    location: str

class CoordinatesOutput(BaseModel):
    latitude: float
    longitude: float

def fetch_coordinates(location: str) -> Dict[str, float]:
    # Function implementation to fetch coordinates
    ...

fetch_coordinates_tool = StructuredTool.from_function(
    func=fetch_coordinates,
    name="fetch_coordinates_tool",
    description="Fetch latitude and longitude for a given location.",
)


In [7]:
import requests
import logging
from typing import Dict, List, Any
from langchain.tools import StructuredTool
from pydantic import BaseModel
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Environment variables
API_KEY = os.environ.get("GOOGLE_MAPS_API")
WEATHER_API_KEY = os.environ.get("OPENWEATHERMAP_API_KEY")

# Structured Input/Output Models
class CoordinatesInput(BaseModel):
    location: str

class CoordinatesOutput(BaseModel):
    latitude: float
    longitude: float

def fetch_coordinates(location: str) -> Dict[str, float]:
    """Fetch coordinates for a given location."""
    try:
        url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={API_KEY}"
        response = requests.get(url).json()
        if response.get("status") == "OK":
            location_data = response["results"][0]["geometry"]["location"]
            return {"latitude": location_data["lat"], "longitude": location_data["lng"]}
        return {}
    except Exception as e:
        logging.error(f"Error fetching coordinates: {e}")
        return {}

fetch_coordinates_tool = StructuredTool.from_function(
    func=fetch_coordinates,
    name="fetch_coordinates_tool",
    description="Fetch latitude and longitude for a given location.",
)

class WeatherInput(BaseModel):
    location: str

class WeatherOutput(BaseModel):
    description: str

def get_weather(location: str) -> str:
    """Fetch weather information for a given location."""
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather"
        params = {"q": location, "appid": WEATHER_API_KEY, "units": "metric"}
        response = requests.get(url, params=params).json()
        if response.get("cod") == 200:
            return f"{response['name']}: {response['weather'][0]['description']} at {response['main']['temp']}°C"
        return "Weather information unavailable."
    except Exception as e:
        logging.error(f"Error fetching weather: {e}")
        return "Weather information unavailable."

weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="weather_tool",
    description="Fetch weather for a given location.",
)

class PlacesInput(BaseModel):
    lat: float
    lng: float
    radius: int
    category: str

class PlacesOutput(BaseModel):
    places: List[Dict[str, Any]]

def fetch_nearby_places(lat: float, lng: float, radius: int, category: str) -> List[Dict[str, Any]]:
    """Fetch nearby places for a given latitude, longitude, radius, and category."""
    try:
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            "location": f"{lat},{lng}",
            "radius": radius * 1000,
            "type": category,
            "key": API_KEY,
        }
        response = requests.get(url, params=params).json()
        if response.get("status") == "OK":
            return response.get("results", [])
        return []
    except Exception as e:
        logging.error(f"Error fetching nearby places: {e}")
        return []

fetch_nearby_places_tool = StructuredTool.from_function(
    func=fetch_nearby_places,
    name="fetch_nearby_places_tool",
    description="Fetch nearby places for given coordinates, radius, and category.",
)

class CountryCurrencyInput(BaseModel):
    lat: float
    lng: float

class CountryCurrencyOutput(BaseModel):
    country: str
    currency: str

def get_country_and_currency(lat: float, lng: float) -> Dict[str, str]:
    """Fetch country and currency details for given latitude and longitude."""
    try:
        url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={API_KEY}"
        response = requests.get(url).json()
        if response.get("status") == "OK":
            country = [comp['long_name'] for comp in response['results'][0]['address_components'] if "country" in comp['types']]
            country_name = country[0] if country else "Unknown"
            return {"country_name": country_name, "currency": "USD"}
        return {"country_name": "Unknown", "currency": "USD"}
    except Exception as e:
        logging.error(f"Error fetching country and currency: {e}")
        return {"country_name": "Unknown", "currency": "USD"}

get_country_and_currency_tool = StructuredTool.from_function(
    func=get_country_and_currency,
    name="get_country_and_currency_tool",
    description="Fetch country and currency for given coordinates.",
)

# Define tools for creating maps and saving itineraries similarly.


### Code Walkthrough

Here's how we set up the Travel Assistant Agent:

1. **Define Tools:**
   We define tools using `StructuredTool` to ensure they accept and return structured data.
   ```python
   tools = [
       fetch_coordinates_tool,
       weather_tool,
       fetch_nearby_places_tool,
       get_country_and_currency_tool,
       create_map_tool,
       save_itinerary_tool,
   ]


### How the Agent Works

The agent uses the provided tools and prompt to execute tasks. Here's how it works:
1. **Reads the Prompt:**
   - The prompt defines the tasks and tools available to the agent.
2. **Analyzes the Input:**
   - The agent processes the user's input and determines the sequence of actions.
3. **Selects Tools Dynamically:**
   - Based on the input, the agent decides which tool to invoke. For example:
     - If the user mentions "weather," the agent uses `weather_tool`.
     - If the user mentions "restaurants near London," the agent uses `fetch_coordinates_tool` and `fetch_nearby_places_tool`.
4. **Executes Tools and Combines Outputs:**
   - The agent executes the selected tools in order and combines their outputs to provide a comprehensive result.


### Agent Setup

In [8]:
# Define tools
tools = [
    fetch_coordinates_tool,
    weather_tool,
    fetch_nearby_places_tool,
    get_country_and_currency_tool,
]

# Initialize LLM and agent
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_template("""
You are a travel assistant. Use the tools below:
1. Fetch coordinates for a location.
2. Get weather details.
3. Fetch nearby places.
4. Fetch country and currency.,
{agent_scratchpad}
""")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


### Error Handling

Agents may encounter errors during execution, such as:
- Invalid inputs (e.g., empty location).
- API failures (e.g., exceeding rate limits or incorrect responses).
- Tool invocation errors.

**How We Handle Errors:**
- Each tool has built-in error handling to log errors and return default outputs when necessary.
- The agent logs all errors encountered during tool execution.
- Users can inspect the logs to debug issues.

Example: If the `fetch_coordinates_tool` fails due to an invalid location, it logs an error and stops further execution for that input.


### Define the Workflow

In [9]:
async def run_workflow(location: str, radius: int, category: str, amount: float):
    print(f"Starting workflow for location: {location}, radius: {radius} km, category: {category}, amount: {amount}")

    # Step 1: Fetch coordinates
    coordinates = await fetch_coordinates_tool.arun({"location": location})
    if not coordinates:
        print("Failed to fetch coordinates.")
        return

    print(f"Coordinates fetched: {coordinates}")

    # Step 2: Fetch weather details
    weather = await weather_tool.arun({"location": location})
    print(f"Weather information: {weather}")

    # Step 3: Fetch nearby places
    places = await fetch_nearby_places_tool.arun({
        "lat": coordinates["latitude"],
        "lng": coordinates["longitude"],
        "radius": radius,
        "category": category
    })
    if not places:
        print("No nearby places found.")
    else:
        print(f"Nearby places: {len(places)} places found.")

    # Step 4: Fetch country and currency
    country_currency = await get_country_and_currency_tool.arun({
        "lat": coordinates["latitude"],
        "lng": coordinates["longitude"]
    })
    print(f"Country and currency details: {country_currency}")

    # Summary
    print("\n=== Workflow Completed ===")
    print(f"Coordinates: {coordinates}")
    print(f"Weather: {weather}")
    print(f"Nearby Places: {places[:3]} (Showing top 3 for brevity)")
    print(f"Country & Currency: {country_currency}")


### Run the Workflow

### Running the Agent

To run the agent and execute the workflow, follow these steps:
1. Ensure all required API keys are set in your environment variables.
2. Execute the following code:
   ```python
   import nest_asyncio
   nest_asyncio.apply()

   # Run the workflow
   await run_workflow("London", radius=5, category="restaurant")


In [10]:
import nest_asyncio
nest_asyncio.apply()

await run_workflow("Hyderabad", radius=5, category="restaurant", amount=5000)


Starting workflow for location: Hyderabad, radius: 5 km, category: restaurant, amount: 5000
Coordinates fetched: {'latitude': 17.406498, 'longitude': 78.47724389999999}
Weather information: Hyderabad: haze at 28.23°C
Nearby places: 20 places found.
Country and currency details: {'country_name': 'India', 'currency': 'USD'}

=== Workflow Completed ===
Coordinates: {'latitude': 17.406498, 'longitude': 78.47724389999999}
Weather: Hyderabad: haze at 28.23°C
Nearby Places: [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 17.4241132, 'lng': 78.4569356}, 'viewport': {'northeast': {'lat': 17.4253438802915, 'lng': 78.4584008302915}, 'southwest': {'lat': 17.4226459197085, 'lng': 78.45570286970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hotel Inner Circle', 'opening_hours': {'open_now': True}, '

### Customizing the Agent

You can customize the agent to fit your needs:
- **Add New Tools:** Define new tools for tasks like fetching hotel prices, booking tickets, or calculating travel times.
- **Modify the Prompt:** Tailor the prompt to include additional instructions or restrict specific tasks.
- **Change the Model:** Switch to a different LLM (e.g., GPT-3.5) if needed.
- **Enhance Error Handling:** Add custom error messages or fallback mechanisms for specific tools.


### Adding More Functionality
#### Tool for Creating Maps and Saving Itineraries

In [11]:
from folium import Map, Marker
from folium.plugins import MarkerCluster
import os
import json

class CreateMapInput(BaseModel):
    lat: float
    lng: float
    places: List[Dict[str, Any]]

class CreateMapOutput(BaseModel):
    map_file: str

def create_map(lat: float, lng: float, places: List[Dict[str, Any]]) -> str:
    """
    Create a map with markers for the provided places and save it as an HTML file.
    """
    try:
        map_object = Map(location=[lat, lng], zoom_start=13)
        marker_cluster = MarkerCluster().add_to(map_object)

        for place in places:
            place_lat = place["geometry"]["location"]["lat"]
            place_lng = place["geometry"]["location"]["lng"]
            name = place.get("name", "Unknown Place")
            Marker([place_lat, place_lng], tooltip=name, popup=place.get("vicinity", "Address not available")).add_to(marker_cluster)

        map_filename = "map_with_places.html"
        map_object.save(map_filename)
        return map_filename
    except Exception as e:
        logging.error(f"Error creating map: {e}")
        return "Map creation failed."

# StructuredTool definition
create_map_tool = StructuredTool.from_function(
    func=create_map,
    name="create_map_tool",
    description="Create a map with markers for the given latitude, longitude, and places."
)


class SaveItineraryInput(BaseModel):
    itinerary: List[Dict[str, Any]]
    map_file: str

class SaveItineraryOutput(BaseModel):
    itinerary_file: str

def save_itinerary(itinerary: List[Dict[str, Any]], map_file: str) -> str:
    """
    Save the trip itinerary and the map file path into a JSON file.
    """
    try:
        data = {"itinerary": itinerary, "map_file": map_file}
        itinerary_filename = "itinerary.json"
        with open(itinerary_filename, "w") as file:
            json.dump(data, file, indent=4)
        return itinerary_filename
    except Exception as e:
        logging.error(f"Error saving itinerary: {e}")
        return "Itinerary save failed."

# StructuredTool definition
save_itinerary_tool = StructuredTool.from_function(
    func=save_itinerary,
    name="save_itinerary_tool",
    description="Save the trip itinerary and map file into a JSON file."
)



### Where Do We Use StructuredTool?

In this notebook, `StructuredTool` is used for the following functionalities:
1. Fetching coordinates (`fetch_coordinates_tool`).
2. Retrieving weather details (`weather_tool`).
3. Finding nearby places (`fetch_nearby_places_tool`).
4. Fetching country and currency (`get_country_and_currency_tool`).
5. Creating a map (`create_map_tool`).
6. Saving the itinerary (`save_itinerary_tool`).

Each of these tools is defined with specific input and output models, ensuring data integrity and seamless agent integration.


### Why Use StructuredTool with Agents?

Agents like LangChain's `create_openai_functions_agent` rely on tools for specific tasks. By using `StructuredTool`:
- The agent knows exactly what data to send and what format to expect in return.
- Complex workflows involving multiple tools (e.g., fetching data from APIs and combining results) become easier to manage.
- Debugging becomes simpler because input and output validation catch issues early.


### Incorporate the Tools into the Agent

In [12]:
# Update tools list to include all tools
tools = [
    fetch_coordinates_tool,
    weather_tool,
    fetch_nearby_places_tool,
    get_country_and_currency_tool,
    create_map_tool,
    save_itinerary_tool,
]

# Agent Setup
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
prompt = ChatPromptTemplate.from_template("""
You are a travel assistant. Follow these steps:
1. Fetch coordinates for a location.
2. Get weather details for the location.
3. Fetch nearby places for the given location.
4. Fetch country and currency based on coordinates.
5. Create a map with the places.
6. Save the itinerary with the map file..
{agent_scratchpad}
""")

# Create the agent
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


### Updated Workflow Execution

In [13]:
async def run_workflow(location: str, radius: int, category: str):
    try:
        # Fetch coordinates
        coordinates = await fetch_coordinates_tool.arun({"location": location})
        if not coordinates:
            raise ValueError("Failed to fetch coordinates.")

        # Fetch weather
        weather = await weather_tool.arun({"location": location})

        # Fetch nearby places
        places = await fetch_nearby_places_tool.arun({
            "lat": coordinates["latitude"],
            "lng": coordinates["longitude"],
            "radius": radius,
            "category": category
        })
        if not places:
            raise ValueError("No nearby places found.")

        # Fetch country and currency
        country_currency = await get_country_and_currency_tool.arun({
            "lat": coordinates["latitude"],
            "lng": coordinates["longitude"]
        })

        # Create a map with nearby places
        map_file = await create_map_tool.arun({
            "lat": coordinates["latitude"],
            "lng": coordinates["longitude"],
            "places": places
        })

        # Save itinerary
        itinerary_file = await save_itinerary_tool.arun({
            "itinerary": [{"name": place["name"], "rating": place.get("rating", "N/A")} for place in places],
            "map_file": map_file
        })

        # Print outputs
        print("Coordinates:", coordinates)
        print("Weather:", weather)
        print("Country & Currency:", country_currency)
        print("Map File:", map_file)
        print("Itinerary File:", itinerary_file)
    except Exception as e:
        logging.error(f"Error during workflow execution: {e}")


### Execute the Workflow

In [14]:
import nest_asyncio
nest_asyncio.apply()

# Run the workflow
await run_workflow("London", radius=5, category="restaurant")


Coordinates: {'latitude': 51.5072178, 'longitude': -0.1275862}
Weather: London: overcast clouds at 4.09°C
Country & Currency: {'country_name': 'United Kingdom', 'currency': 'USD'}
Map File: map_with_places.html
Itinerary File: itinerary.json


#### Advanced Usage of StructuredTool
If you want to add more depth, include advanced topics:
* Using StructuredTool for API chaining: Explain how one tool's output can be
used as the input for another.
* Combining multiple StructuredTools in a single task.
* Creating custom validation logic in pydantic models (e.g., ensuring radius is non-negative or places is non-empty).